### Caricamento dei file
Con tale spezzone di codice, si vanno a caricare all'interno della cartella di colab i file:
- libs/auffusion.py
- libs/convertet.py
- libs/auffusion_converter.py
- libs/spettrogram.py

Una volta caricati questi file l'ambiente è pronto per partire

In [1]:
from google.colab import files

uploaded = files.upload()


Saving auffusion.py to auffusion.py
Saving spettrogram.py to spettrogram.py
Saving converter.py to converter.py
Saving auffusion_converter.py to auffusion_converter.py


### Rifinitura dell'ambiente
Tramite questo comando di vanno ad inserire:
- .project-root: che permette alla libreria rootutils di poter trovare la root del programma
- creazione della cartella outputs, che raccoglie i risultati che vengono prodotti
- creazione della cartella suoni in cui poter inserire i suoni che il sistema può usare come riferimento

In [2]:
!touch "/content/.project-root"
!mkdir "/content/outputs"
!mkdir "/content/suoni"

### Libreria da installare
Nel caso di utilizzo dell'hardware GPU T4 di colab, l'ambiente sarà perfettamente configurato e bisognerà solo installare ed aggiornare le seguenti librerie per evitare errori di dipendenza

In [3]:
!pip install lightning
!pip install --upgrade huggingface_hub
!pip install rootutils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Comando di liberazione della memoria
Nel caso in cui il sistema si interrompa prima di terminare, o per eventuali errori o per interruzioni da parte dell'utente, è consigliato utilizzare tale comando, per "liberare" la memoria da eventuali residui non deallocati dal programma per via dell'errore riscontrato

In [13]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

117

### Codice di emergenza
Nel caso ci siano conflitti senza senso, a cui magari le dipendenze sono risolte e il codice è di buona qualità, può essere buono "riavviare" la sessione, tramite tali comandi si riavvia la sessione, ma non si perdono i file inseriti sino ad ora (è differente dalla semplice disconnessione di colab, poichè i file rimangono persistenti e non si perdono)

In [ ]:
# import os
# os.kill(os.getpid(), 9)

### Aggiornamento dei file di libreria
Dato che spesso ci capitava di aggiornare su colab i file di libreria per le eventuali prove, di seguito vi è il codice per prevederne il cambiamento. In questo caso ristretto ai casi spettrogram e auffusion (libreria più cambiata)

In [7]:
import importlib
import sys

# Se già importata, rimuovila dal sys.modules per sicurezza
if 'libs.auffusion' in sys.modules:
    del sys.modules['libs.auffusion']

# Ora importa normalmente
import libs.auffusion

# E ricaricala per sicurezza
importlib.reload(libs.auffusion)

# Se già importata, rimuovila dal sys.modules per sicurezza
if 'libs.spettrogram' in sys.modules:
    del sys.modules['libs.spettrogram']

# Ora importa normalmente
import libs.spettrogram

# E ricaricala per sicurezza
importlib.reload(libs.spettrogram)


<module 'libs.spettrogram' from '/content/libs/spettrogram.py'>

### Funzione per l'esportazione dei risultati su google drive

In [6]:
import shutil
import os
from google.colab import drive

def copy_to_drive(src_folder_path, dest_folder_name=None):
    """
    Copia una cartella dall'ambiente Colab al Drive, nella cartella 'outputs'.
    Se 'outputs' non esiste, la crea.

    Args:
        src_folder_path (str): percorso della cartella da copiare (es. /content/my_folder)
        dest_folder_name (str): nome con cui copiare la cartella su Drive (default: stesso nome di src)
    """
    # Monta Google Drive
    drive.mount('/content/drive', force_remount=True)

    # Percorso della cartella "outputs" in Drive
    drive_outputs_path = '/content/drive/MyDrive/outputs'

    # Crea la cartella outputs se non esiste
    if not os.path.exists(drive_outputs_path):
        os.makedirs(drive_outputs_path)
        print(f"Cartella 'outputs' creata in Drive.")

    # Nome di destinazione (se non specificato, usa il nome originale)
    folder_name = dest_folder_name or os.path.basename(src_folder_path.rstrip('/'))

    # Percorso completo di destinazione
    dest_path = os.path.join(drive_outputs_path, folder_name)

    # Se la cartella già esiste in Drive, rimuoverla prima (opzionale)
    if os.path.exists(dest_path):
        print(f"Cartella '{folder_name}' già presente in Drive. Sovrascrivo.")
        shutil.rmtree(dest_path)

    # Copia la cartella
    shutil.copytree(src_folder_path, dest_path)
    print(f"Cartella '{folder_name}' copiata con successo in Drive.")


### Codice effettivo di utilizzo della rete implementata
Tale codice è quello che viene utilizzati in fase di generazione, prima di lanciarlo si consiglia di settare i seguenti parametri:

#### Prompts
Stringhe di testo con cui si può descrivere il risultato che si vuole ottenere o il contenuto informativo che il risultato deve contenere:
- prompt_audio: prompt di riferimento per la generazione dell'audio
- prompt_video: prompt di riferimento per la generazione dell'immagine

#### Guidance Scale
Il guidance scale permette di impostare, in base al valore impostato, se un elemento deve essere correlato di più al prompt o meno. In generale i valori di tali parametri vanno da un minimo di 5 (minore dipendenza dal prompt) ad un massimo di 15 (maggiore dipendenza dal prompt)

#### Path di riferimento
- ref_path: è il path per l'audio da considerare come riferimento per la generazione
- output_path: è il riferimento alla cartella in cui saranno salvati i vari elementi

Il funzionamento è semplice, dopo aver scaricato le dipendenze, lo script genera 12 elementi e li inserisce all'interno della output_path con dei nomi che sono estratti da due parole del prompt

In [12]:
# Librerie e loro gestione

# Ora puoi accedere a tutto come auffusion.AuffusionGuidance ecc.
from libs.auffusion import AuffusionGuidance
import matplotlib.pyplot as plt
import torch
import numpy as np
import soundfile as sf
from libs.spettrogram import *
import gc
import os
import json

if __name__=="__main__":
    # -----> [PROMPTS] <----- #
    # Prompt per la generazione dell'audio
    prompt_audio = "birds singing"

    # Prompt per la generazione dell'immagine
    prompt_video = "a painting of snowy mountains, grayscale"

    # -----> [GUIDANCE SCALE] <----- #
    # Guidance scale associato all'audio
    guidance_scale_audio=8.0

    # Guidance scale associato all'immagine
    guidance_scale_video=10.0

    # -----> [DIPENDENZA DAL RIFERIMENTO] <----- #
    # Variabile utilizzata per specificare la dipendenza rispetto allo spettrogramma dato in ingresso
    strength = 0.8

    # -----> [PATH DI RIFERIMENTO] <----- #
    # Riferimento all'elemento di riferimento (mettere a None se non si vuole influenza esterna)
    ref_audio="/content/suoni/prime-facts7-nature-sound-mornings-291488.mp3"

    # Riferimento alla cartella in cui salvare i valori generati dai successivi passi
    output_path="/content/outputs"

    # -----> [VARIABILI ORGANIZZATIVE] <----- #
    batch_size = 12
    name_in_drive = name_audio = prompt_audio.split(' ')[0] + '_' + prompt_audio.split(' ')[1]

    # Creazione del dizionario con le informazioni
    config_data = {
        "prompt_audio": prompt_audio,
        "prompt_video": prompt_video,
        "guidance_scale_audio": guidance_scale_audio,
        "guidance_scale_video": guidance_scale_video,
        "ref_audio": ref_audio,
        "strength": strength
    }

    # Verifica della presenza della cartella di outputs
    os.makedirs(output_path, exist_ok=True)

    # Percorso del file JSON
    json_path = os.path.join(output_path, "config.json")

    # Scrittura su file JSON
    with open(json_path, "w") as f:
        json.dump(config_data, f, indent=4)

    for n in range(0, batch_size):

        # Dichiarazione del modello
        model = AuffusionGuidance(fp16=True)

        # Inserimento del modello sulla GPU
        model = model.to(torch.device('cuda'))

        # Preprocessing eventuale del suono (se inserito)
        if ref_audio is not None:
            # Nel caso in cui sia inserito, si va a ricavare lo spettrogramma
            input_spectrogram, shape = spectrify(ref_audio)
        else:
            # Altrimenti si lascia a None
            input_spectrogram = None

        # Inizio generazione delle immagini
        print(f"[MAIN]\t-\tGenerando la: [{n}]")

        try:
            # Funzione di classe per la generazione dello spettrogramma
            spect = model.prompt_to_spec(prompt_audio, prompt_video,
                                    height=256, width=1024, num_inference_steps=100,
                                    device=torch.device('cuda'), guidance_scale_video=guidance_scale_video, input_spectrogram=input_spectrogram,
                                    guidance_scale_audio=guidance_scale_audio, strength=strength)
        except RuntimeError as e:
            print(e)

        # Printi di eventuali statistiche interessanti
        print(torch.mean(spect), torch.min(spect), torch.max(spect))

        # Inizio di post-processing per salvare l'audio e l'immagine
        # Ricavo l'immagine generata
        img = spect.detach().cpu().squeeze(0)
        img = img.permute(1, 2, 0).numpy()
        img = img.clip(0, 1)
        imgs = (img * 255).round().astype('uint8')

        # RIcavo gli elementi con cui costruire i path di salvataggio
        name_audio = prompt_audio.split(' ')[0] + '_' + prompt_audio.split(' ')[1]
        name_video = prompt_video.split(' ')[4] + '_' + prompt_video.split(' ')[5]

        # Converto lo spettrogramma in audio tramite una funzione di classe
        audio = model.spec_to_audio(spect.squeeze(0))

        # Salvo l'audio
        sf.write(f'{output_path}/{name_audio}_{n}.wav', np.ravel(audio), samplerate=16000)

        # Salvataggio dell'immagine
        plt.figure()
        plt.imsave(f'{output_path}/{name_video}_{n}.png', img)

        # Liberazione della GPU dalla classe caricata
        torch.cuda.empty_cache()
        gc.collect()

    copy_to_drive("/content/outputs", name_in_drive)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--auffusion--auffusion-full-no-adapter/snapshots/a75ebf2e22d76f8b3cdafcd2a42fd4abba43f7ec/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 100000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


KeyboardInterrupt: 

In [10]:
copy_to_drive("/content/outputs", name_in_drive)

Mounted at /content/drive
Cartella 'outputs' creata in Drive.
Cartella 'birds_singing' copiata con successo in Drive.


In [ ]:
cmd_wav = f'cp "/content/{name_audio}.wav" "/content/drive/MyDrive/Progetto_ESM/Prove/{name_audio}.wav"'
cmd_png = f'cp "/content/{name_audio}.png" "/content/drive/MyDrive/Progetto_ESM/Prove/{name_audio}.png"'

!{cmd_wav}
!{cmd_png}